In [13]:
import os
import sys
import numpy as np
import pandas as pd
import pprint as pp
from pathlib import Path

sys.path.append(os.path.abspath(".."))

from utils.model_transformer.model import Model
from utils.model_ann.ann import build_index, search

device = "cpu"

# Load Dataset

In [14]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])

In [15]:
dataset_dir = Path(base_dir).joinpath(f"data/dataset/{DOMAIN}/interactions")
df = pd.read_parquet(dataset_dir)
df.head()

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3507,2025-09-21 03:01:00,click,46,F,슬림핏 실루엣의 데님 오버롤,레드,스포티,슬림핏,데님,겨울,롱슬리브,오버롤
1,1,9867,2025-09-21 03:02:00,click,46,F,봄 슬림핏 레드 나일론 롱슬리브 오버롤,레드,스포티,슬림핏,나일론,봄,롱슬리브,오버롤
2,1,3471,2025-09-02 00:05:00,click,46,F,"간절기 시즌, 레트로 무드의 오렌지 퍼 롱슬리브 루즈핏 원피스",오렌지,레트로,루즈핏,퍼,간절기,롱슬리브,원피스
3,1,6462,2025-09-02 00:09:00,click,46,F,"트렌디한 레트로룩, 슬림핏 원피스 in 여름",오렌지,레트로,슬림핏,스웨이드,여름,숏슬리브,원피스
4,1,441,2025-09-19 08:51:00,click,46,F,"봄 시즌, 캐주얼 무드의 와인 스웨이드 롱슬리브 루즈핏 슬랙스",와인,캐주얼,루즈핏,스웨이드,봄,롱슬리브,슬랙스


# Load Model

In [16]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])
model_dir = Path(base_dir).joinpath(f"data/model/{DOMAIN}")

In [17]:
model = Model(model_dir=model_dir)

/Users/hwangyungwan/miniforge3/envs/ds_p313/lib/python3.13/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


# Prediction Test

In [18]:
body = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
        ],
    }
]

pp.pprint(model.predict(body))

[{'item_vector': [0.011732138693332672,
                  0.018032588064670563,
                  -0.016697775572538376,
                  0.020273707807064056,
                  0.021498695015907288,
                  -0.02249741554260254,
                  -0.021672751754522324,
                  -0.0377618744969368,
                  0.024913884699344635,
                  0.026755601167678833,
                  -0.02478354424238205,
                  -0.0058990418910980225,
                  0.16772346198558807,
                  0.027432259172201157,
                  -0.038914766162633896,
                  -0.026893820613622665,
                  0.0015498995780944824,
                  -0.026692137122154236,
                  0.07795572280883789,
                  0.008195996284484863,
                  0.04019976407289505,
                  -0.009768154472112656,
                  0.049679502844810486,
                  -0.02600354701280594,
                  0.025370564311742

# Recommendation

In [19]:
item_metadata_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/item_metadata.parquet"
)
df_item = pd.read_parquet(item_metadata_path)

text_vectors_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/text_vectors.parquet"
)
df_text_vectors = pd.read_parquet(text_vectors_path)

In [20]:
text_vectors = np.array(df_text_vectors["text_vector"].tolist())
item_vectors = np.array(text_vectors).astype(np.float32)
item_id_maps = dict(zip(df_text_vectors["item_id"].index, df_text_vectors["item_id"]))

In [21]:
# item_vectors: (num_items, dim) numpy float32 배열이라고 가정
item_index = build_index(item_vectors)
item_idx = 0
query_vector = item_vectors[item_idx]
result = search(query_vector, item_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 1.0000001192092896, 6140: 0.9872053265571594, 8858: 0.9866687655448914, 9990: 0.9789416193962097, 4424: 0.978618860244751}


In [22]:
body = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
        ],
    }
]

prediction = model.predict(body)[0]

In [23]:
user_idx = 0
query_vector = np.array(prediction["item_vector"])
result = search(query_vector, item_index, top_k=30)

item_indies = list(result.keys())[1:]

recommendation = {}
for i in item_indies:
    item_id = item_id_maps[i]
    recommendation[item_id] = result[i]

recommendation

{6783: 0.9753272533416748,
 9696: 0.9746831059455872,
 5549: 0.9744634628295898,
 7951: 0.9743757843971252,
 4364: 0.9741804003715515,
 9125: 0.9740840196609497,
 8412: 0.9740766286849976,
 5137: 0.9740387201309204,
 8314: 0.9739794135093689,
 1180: 0.9735918641090393,
 4390: 0.973501980304718,
 9104: 0.9734668731689453,
 5126: 0.9734300971031189,
 3979: 0.9733468294143677,
 1241: 0.9733180999755859,
 818: 0.9731886982917786,
 6060: 0.9731830358505249,
 3857: 0.9731447696685791,
 2745: 0.973118245601654,
 5308: 0.9731072187423706,
 7700: 0.9730779528617859,
 1185: 0.9730347990989685,
 779: 0.9730024337768555,
 1448: 0.9728865623474121,
 8974: 0.9728692770004272,
 1207: 0.9727843403816223,
 5664: 0.9727834463119507,
 5058: 0.9726940989494324,
 4374: 0.9726658463478088}

In [24]:
df_recommendation = df_item[
    df_item["item_id"].isin(list(recommendation.keys()))
].reset_index(drop=True)

df_recommendation

,item_id,title,color,style,fit,material,season,sleeve,category
0,779,간절기 감성 루즈핏 스웨이드 니트,블랙,포멀,루즈핏,스웨이드,간절기,숏,니트
1,818,가을 감성 오버핏 모달 재킷,그레이,러블리,오버핏,모달,가을,숏슬리브,재킷
2,1180,가을 루즈핏 카키 린넨 숏 후드티,카키,시크,루즈핏,린넨,가을,숏,후드티
3,1185,봄 감성 슬림핏 모달 스커트,올리브,스포티,슬림핏,모달,봄,7부,스커트
4,1207,겨울 크롭핏 브라운 모달 롱 스커트,브라운,미니멀,크롭핏,모달,겨울,롱,스커트
5,1241,겨울 감성 루즈핏 데님 재킷,레드,스포티,루즈핏,데님,겨울,숏슬리브,재킷
6,1448,겨울 감성 루즈핏 메쉬 코트,오렌지,미니멀,루즈핏,메쉬,겨울,숏슬리브,코트
7,2745,겨울 감성 루즈핏 스웨이드 니트,와인,러블리,루즈핏,스웨이드,겨울,롱슬리브,니트
8,3857,겨울 감성 레귤러핏 폴리 후드티,베이지,시크,레귤러핏,폴리,겨울,숏슬리브,후드티
9,3979,간절기 감성 루즈핏 스웨이드 재킷,카키,댄디,루즈핏,스웨이드,간절기,숏,재킷
